<a href="https://colab.research.google.com/github/positivejmk/2021-2-practice/blob/master/%EA%B3%A0%EA%B0%9D%EB%B3%84_%EA%B1%B0%EB%9E%98%EC%9D%BC%EC%9E%90_%EC%B5%9C%EC%B4%88%2C_%EC%B5%9C%EA%B7%BC_%EB%82%A0%EC%A7%9C%EC%97%90_%EB%94%B0%EB%9D%BC_%EA%B1%B0%EB%9E%98%EA%B8%B0%EA%B0%84_%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os
import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2021년(제2회) NH투자증권 빅데이터 경진대회/nh prediction.csv")

In [20]:
df.head()

,Unnamed: 0,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,1,9,3,2,2,9,5,5,8,101,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0
1,6,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,80,1,9,3,2,2,9,5,5,8,1361,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
2,15,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,5,1,9,3,2,2,9,5,5,8,2530,12,2,99,20180517,786.0,14619600.0,1000.0,18600.0
3,18,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,22,1,9,3,2,2,9,5,5,8,1969,8,2,1,20201112,60.0,462000.0,500.0,7700.0
4,20,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,324,1,9,3,2,2,9,5,5,8,1696,10,3,99,20180905,183.0,8125200.0,5000.0,44400.0


In [26]:
#먼저 이미 datetime으로 바꾸는데, 정수형으로 바꾸고 싶을 때 (정수형일때 min, max값 찾기 용이)
#df['byn_dt'] = df['byn_dt'].astype(str)
#df['byn_dt'] = df.byn_dt.str.replace('-','').astype(int)

In [22]:
df['byn_dt'] = pd.to_datetime(df['byn_dt'])

In [25]:
df1 = df.groupby('act_id')['byn_dt'].min()
df1 = pd.DataFrame(df1)

max = df.groupby('act_id')['byn_dt'].max()
max = pd.DataFrame(max)

df1['max'] = max['byn_dt']
df1 = df1.rename(columns={'byn_dt':'min'})

df1['duration'] = df1['max'] - df1['min']

In [24]:
df1

,min,max,duration
act_id,,,
0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c5467ece45d5b35e99912a,2019-03-20,2020-09-28,558 days
0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa9b6e6efa399d0bdf31,2020-03-26,2020-12-02,251 days
0008d1bfac569bdb998897e2eb8519dec06b9277a686f4c6e06de582c9791b35,2019-01-03,2020-10-27,663 days
000a83a961cbcbce40673aed998effbd462fb10dcac8e64c6fa720f2150d3f5a,2018-11-06,2020-03-02,482 days
000b92ca83a96312a0909b1f489340f4d399c86555012ffe2c41952ae43dc741,2018-07-25,2018-11-12,110 days
...,...,...,...
ffdccfb9dc24cf41843f6e7f34704ccb73ac81acc29aa39b043fe7f90959e5f3,2018-11-29,2020-12-21,753 days
ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7bace472b4365f5028535,2019-06-21,2020-10-13,480 days
ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae96e7cd290e431aee0a,2019-01-22,2020-08-20,576 days
